# ELEC3305 - Real Time Sonar App


In [11]:
# Import functions and libraries
import numpy as np
from scipy import signal
from numpy import *
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
from rtsonar import rtsonar
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Task III: Play with different parameters with the real-time sonar!

The function `rtsonar()` in `rtsonar.py` provides a wrapper to create a real-time sonar plot using user-defined processing functions. Copy and paste your functions genPulseTrain(), genChirpPulse(), crossCorr(), findDelay(), and dist2time() to the code cell below. Scroll down to the bottom and you should be able to run `rtsonar()`!

The detection resolution and sensitivity will depend on the parameters you will choose for the sonar. The longer the pulse is, the worse time resolution you will get, but much stronger matched filtering amplitude. You can buy the time resolution by using pulse compression with chirp pulses at the expense of increasing the bandwidth. 

* Run the sonar with different parameters. You can get interesting targets by moving your laptop, moving yourself next to the computer, using a book as a reflecting surface. It would be easier for you to distinguish the target if you move it back and forth. Play with the pulse lengths, the frequency sweeps, etc. 
* To get the best real-time performance, you should restart the kernel every time you run this lab. 
* Submit 4 interesting set of parameters. Explain why you chose them. 




In [13]:
# Copy and paste your 5 functions here:
# genChirpPulse()
def genChirpPulse(Npulse, f0, f1, fs):
    #     Function generates an analytic function of a chirp pulse
    #     Inputs:
    #             Npulse - pulse length in samples
    #             f0     - starting frequency of chirp
    #             f1     - end frequency of chirp
    #             fs     - sampling frequency
    #     Output:
    #              pulse - chirp pulse
    t = np.r_[0:Npulse/fs:1/fs] # time index
    T = Npulse/fs # period
    
    f_of_t = f0 + ((f1-f0)/(2*T))*t # f of t inside phi of t
    chirpPulse = np.exp(1j*2*np.pi*f_of_t*t) # create the chirp signal
    return chirpPulse



    
# genPulseTrain()
def genPulseTrain(pulse, Nrep, Nseg):
    # Funtion generates a pulse train from a pulse. 
    #Inputs:
    #    pulse = the pulse generated by genChirpPulse
    #    Nrep  =  number of pulse repetitions
    #    Nseg  =  Length of pulse segment >= length(pulse)
    pulse = np.append(pulse, np.zeros(Nseg - len(pulse))) # adding zeros to the pulse matrix
    
    
    ptrain = np.tile(pulse, Nrep) # get Nrepetition of the pulse
    return ptrain

    
# crossCorr()
def crossCorr( rcv, pulse_a ):
    # Funtion generates cross-correlation between rcv and pulse_a
    # Inputs:
    #    rcv - received signal
    #    pulse_a - analytic pulse
    # Output:
    #    Xrcv - cross-correlation between rcv and pulse_a
    
    Xrcv = signal.fftconvolve(rcv, pulse_a[::-1], mode = "full")
    
    #width,height = plt.figaspect(0.3)
    #plt.figure(figsize=(width,height))
    #plt.plot(np.r_[0:len(Xrcv)],np.real(Xrcv))
    #plt.title("Cross Correlation of the Received Signal with the Analytic Function of the Pulse (Matched Filter)")
    
    return Xrcv
    


# findDelay()
def findDelay(Xrcv, Nseg):
    # finds the first pulse
    # Inputs:  
    #         Xrcv - the received matched filtered signal
    #         Nseg - length of a segment
    # Output:
    #          idx - index of the beginning of the first pulse
    
    return np.argmax(Xrcv[0:Nseg]) # returns the index of the maximum value
    


# dist2time()
def dist2time( dist, temperature=21):
    # Converts distance in cm to time in secs
    # Inputs:
    # dist        - distance to object in cm
    # temperature - in celcius
    # Output:
    # t           - time in seconds between transmitted pulse and echo
    velocity_of_sound = 331.5*sqrt(1+temperature/273.15)* 100 # unit is in cm/s
    time = (dist/velocity_of_sound)*2 # include target and back
    
    return time # in seconds

In [ ]:
# Run this for Real-time Sonar
# Change the parameters!
# Your functions will need to be efficient for this to work. 
# If you get nothing try thinking about how to speed up your code.
fs = 48000 # Sampling frequency
f0 = 6000 # Chirp initial frequency
f1 = 12000 # Chirp ending frequency
DurationS = 0.00001 # s
Pause = 0.1 # s

#Npulse = int(fs * DurationS)
Npulse = 500 # 500 Length of Chirp Pulse
Nrep = 24 # 24 Number of repetition in a pulse train (determines vertical length of plot )
Nseg = int(fs * Pause)
#Nseg = 2048*2 # Number of samples between pulses (determines maximum time-of-arrival)
Nplot = 1024 # Number of pixels in plot along horizontal direction (higher resolution is slower)
maxdist = 200 # Maximum distance in cm
temperature = 20 # Temperature     


functions = (genChirpPulse, genPulseTrain, crossCorr, findDelay, dist2time) #join the functions together

stop_flag = rtsonar( f0, f1, fs, Npulse, Nseg, Nrep, Nplot, maxdist, temperature, functions )

In [4]:
# Run this to stop the sonar (it will take a few seconds to stop)
stop_flag.set()

## BELOW ARE THE CORRECT FUNCTION FOR rtsonar TO GET IT WORKING! TRUST ME!

  
  ## def rtsonar( f0, f1, fs, Npulse, Nseg, Nrep, Nplot, maxdist, temperature, functions ):

    clear_output();
    genChirpPulse = functions[0]
    genPulseTrain = functions[1]
    
    pulse_a = genChirpPulse(Npulse, f0,f1,fs)
    hanWin = np.hanning(Npulse)
    # hanWin = np.reshape(hanWin, (Npulse,1) ) <- I commented that line and its working !!! Cheers !!!
    pulse_a = np.multiply(pulse_a,hanWin)
    pulse = np.real(pulse_a)
    ptrain = genPulseTrain(pulse, Nrep, Nseg)
    
    # create an input output FIFO queues
    Qin = queue.Queue()
    Qout = queue.Queue()
    Qdata = queue.Queue()

    # create a pyaudio object
    p = pyaudio.PyAudio()
    
    # create black image
    img = np.zeros((Nrep,Nplot), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((Nrep, Nplot, 4))
    view[:,:,3] = 255;
    
    # initialize plot
    fig = bk.figure(title = 'Sonar',  y_axis_label = "Time [s]", x_axis_label = "Distance [cm]",
                    x_range=(0, maxdist), y_range=(0, Nrep * Nseg / fs ) , 
                    plot_height = 400, plot_width = 800 )
    fig.image_rgba( image = [ img ], x=[0], y=[0], dw=[maxdist], dh=[Nrep * Nseg / fs ], name = 'echos' )
    bk.show(fig,notebook_handle=True) # add notebook_handle=True to make it update

    # initialize stop_flag
    stop_flag = threading.Event()

    # initialize threads
    t_put_data = threading.Thread(target = put_data,   args = (Qout, ptrain, Nseg / fs*3, stop_flag  ))
    t_rec = threading.Thread(target = record_audio,   args = (Qin, p, fs, stop_flag  ))
    t_play_audio = threading.Thread(target = play_audio,   args = (Qout, p, fs, stop_flag  ))
    t_signal_process = threading.Thread(target = signal_process, args = ( Qin, Qdata, pulse_a, Nseg, Nplot, fs, maxdist, temperature, functions, stop_flag))
    t_image_update = threading.Thread(target = image_update, args = (Qdata, fig, Nrep, Nplot, stop_flag ) )

    # start threads
    t_put_data.start()
    t_rec.start()
    #record_audio(Qin, p, fs, stop_flag)
    t_play_audio.start()
    t_signal_process.start()
    t_image_update.start()

    return stop_flag
    